# 11/17 ゼミ

*シミュレーション*  
* 真の区間重要度に対してA, B, C, D, E 各手法で1000ずつ一対比較行列を取る  
* 1000個の区間AHPを解き、それぞれの指標を算出した  

|N=5|EV|A|B|C|D|E|
|:----|:----|:----|:----|:----|:----|:----|
| |P_ave| |0.44319|0.39936|0.42855|0.44626|
| |Q_ave| |0.86888|0.91659|0.89502|0.87318|
| |R_ave| |0.48545|0.41916|0.45947|0.48897|
| |S_ave| |0.59332|0.54614|0.57781|0.59569|


|N=5|EV|A|B|C|D|E|
|:----|:----|:----|:----|:----|:----|:----|
| |P_var| |0.03131|0.03542|0.0328|0.03205|
| |Q_var| |0.03847|0.0319|0.03468|0.03881|
| |R_var| |0.04308|0.04262|0.04323|0.04427|
| |S_var| |0.02986|0.03456|0.03155|0.03096|

実験に用いたコードは以下の通り  

In [58]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [59]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./src/interval-weight-estimation.jl")
include("./src/display/display-crisp-matrix.jl")
include("./src/display/display-interval-vector.jl")
include("./src/display/display-vector.jl")
include("./src/crisp-pcm.jl")
include("./src/importance-estimation.jl")
include("./src/solve-interval-ahp.jl")

solveIntervalAHP (generic function with 1 method)

In [60]:
# 評価基準数
N = 5;

In [61]:
# 真の区間重要度を読み込み
df = CSV.File("./src/data/N5_Given_interval_weight.csv", header=false) |> DataFrame
T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
for i in 1:N
    push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
end

display(L"T = %$(intervalVectorLaTeXString(T))")

L"$T = \begin{pmatrix} \left[ 0.27, 0.33 \right] \\ \left[ 0.21, 0.29 \right] \\ \left[ 0.15, 0.25 \right] \\ \left[ 0.09, 0.21 \right] \\ \left[ 0.03, 0.17 \right] \end{pmatrix}$"

In [62]:
# 一対比較行列を1000個読み込み
df1 = CSV.File("./src/data/N5_a3_A_PCM_int.csv", header=false) |> DataFrame

# データフレームをN×Nのサブデータフレームに分割
subdfs = split_dataframe(df1, N);

In [63]:
# 結果格納用
P = Matrix{Float64}(undef, N, 1000)
Q = Matrix{Float64}(undef, N, 1000)
R = Matrix{Float64}(undef, N, 1000)
F = Matrix{Float64}(undef, N, 1000)

# 1000個のデータで推定
for (i, subdf) in enumerate(subdfs)

    # 一対比較行列を取り出す
    A = Matrix(subdf)
    # display(L"A = %$(matrixLaTeXString(A))")

    # 提案手法で区間重要度推定
    solution = solveIntervalAHP(A, AD)
    E = solution.W # 区間重要度の推定値
    # display(L"E = %$(intervalVectorLaTeXString(E))")
    # display(L"T = %$(intervalVectorLaTeXString(T))")

    TcapE = T .∩ E # 真の区間重要度と推定値の積集合
    TcupE = T .∪ E # 真の区間重要度と推定値の和集合
    # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
    # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
    
    # P, Q, R, F の計算
    P[:, i] = calculate_P(T, E) # 真の区間重要度と推定値の積集合の割合
    Q[:, i] = calculate_Q(T, E) # 真の区間重要度と推定値の積集合の割合
    R[:, i] = calculate_R(T, E) # 真の区間重要度と推定値の積集合の割合
    F[:, i] = calculate_F(T, E) # F値
    display(L"P = %$(VectorLaTeXString(P[:,i]))")
    display(L"Q = %$(VectorLaTeXString(Q[:,i]))")
    display(L"R = %$(VectorLaTeXString(R[:,i]))")
    display(L"F = %$(VectorLaTeXString(F[:,i]))")
end

L"$P = \begin{pmatrix} 0.175 \\ 0.377 \\ 0.413 \\ 0.323 \\ 0.513 \end{pmatrix}$"

L"$Q = \begin{pmatrix} 1.0 \\ 1.0 \\ 1.0 \\ 1.0 \\ 1.0 \end{pmatrix}$"

L"$R = \begin{pmatrix} 0.175 \\ 0.377 \\ 0.413 \\ 0.323 \\ 0.513 \end{pmatrix}$"

L"$F = \begin{pmatrix} 0.298 \\ 0.548 \\ 0.584 \\ 0.488 \\ 0.678 \end{pmatrix}$"

L"$P = \begin{pmatrix} 0.178 \\ 0.307 \\ 0.341 \\ 0.212 \\ 0.364 \end{pmatrix}$"

...

L"$F = \begin{pmatrix} 0.217 \\ 0.462 \\ 0.368 \\ 0.489 \\ 0.758 \end{pmatrix}$"

L"$P = \begin{pmatrix} 0.279 \\ 0.2 \\ 0.25 \\ 0.237 \\ 0.427 \end{pmatrix}$"

L"$Q = \begin{pmatrix} 1.0 \\ 1.0 \\ 1.0 \\ 0.872 \\ 1.0 \end{pmatrix}$"

L"$R = \begin{pmatrix} 0.279 \\ 0.2 \\ 0.25 \\ 0.245 \\ 0.427 \end{pmatrix}$"

L"$F = \begin{pmatrix} 0.436 \\ 0.333 \\ 0.4 \\ 0.383 \\ 0.598 \end{pmatrix}$"

In [64]:
P_ave = mean(P)
Q_ave = mean(Q)
R_ave = mean(R)
F_ave = mean(F)
display(L"Pₐᵥₑ = %$(round(P_ave, digits=5))")
display(L"Qₐᵥₑ = %$(round(Q_ave, digits=5))")
display(L"Rₐᵥₑ = %$(round(R_ave, digits=5))")
display(L"Fₐᵥₑ = %$(round(F_ave, digits=5))")

L"$Pₐᵥₑ = 0.33681$"

L"$Qₐᵥₑ = 0.93967$"

L"$Rₐᵥₑ = 0.34803$"

L"$Fₐᵥₑ = 0.48343$"

In [65]:
P_var = var(P)
Q_var = var(Q)
R_var = var(R)
F_var = var(F)
display(L"Pᵥₐᵣ = %$(round(P_var, digits=5))")
display(L"Qᵥₐᵣ = %$(round(Q_var, digits=5))")
display(L"Rᵥₐᵣ = %$(round(R_var, digits=5))")
display(L"Fᵥₐᵣ = %$(round(F_var, digits=5))")

L"$Pᵥₐᵣ = 0.02606$"

L"$Qᵥₐᵣ = 0.0272$"

L"$Rᵥₐᵣ = 0.03129$"

L"$Fᵥₐᵣ = 0.02953$"